In [12]:
# ==========================================
# 🚀 BEHAVIOR DETECTOR - TRAINING SCRIPT
# ==========================================
!pip install -q ultralytics roboflow
import os, shutil, yaml, random
from roboflow import Roboflow
from glob import glob
from ultralytics import YOLO


In [16]:
# ==========================================
# 1. CONFIG
# ==========================================
API_KEY = "YPl8sjuu0ikszmZbb3yH"

PHONE_DS_ID    = "learning-mmxpm/phone-detection-tqu3x"
FOOD_DS_ID     = "smartlight/eating-person-models"
DRINK_DS_ID    = "ai-bt7mn/drinking-huzef"
PERSONAL_DS_ID = "student-b3qgz/behaviordetector"

MAX_IMAGES = 200
FINAL_DIR = "/content/merged_dataset"
random.seed(42)

TARGET_CLASSES = {
    0: ['phone', 'mobile', 'cell', 'smart'],
    1: ['food', 'eat', 'burger', 'sandwich'],
    2: ['drink', 'cup', 'bottle', 'water']
}

In [17]:
# ==========================================
# 2. DOWNLOAD & PREPARE
# ==========================================
rf = Roboflow(api_key=API_KEY)

# Setup folders
for split in ['train', 'valid']:
    os.makedirs(os.path.join(FINAL_DIR, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(FINAL_DIR, split, 'labels'), exist_ok=True)

def get_dataset(dataset_id):
    """Downloads dataset and returns the path."""
    try:
        print(f"⬇️ Downloading {dataset_id}...")
        parts = dataset_id.split("/")
        project = rf.workspace(parts[0]).project(parts[1])
        # Download version 1 (or latest available)
        dataset = project.version(project.versions()[0].version).download("yolov8")
        return dataset.location
    except Exception as e:
        print(f"❌ Error downloading {dataset_id}: {e}")
        return None

path_phone = get_dataset(PHONE_DS_ID)
path_food = get_dataset(FOOD_DS_ID)
path_drink = get_dataset(DRINK_DS_ID)
path_personal = get_dataset(PERSONAL_DS_ID)


⬇️ Downloading learning-mmxpm/phone-detection-tqu3x...
loading Roboflow workspace...
loading Roboflow project...
⬇️ Downloading smartlight/eating-person-models...
loading Roboflow workspace...
loading Roboflow project...
⬇️ Downloading ai-bt7mn/drinking-huzef...
loading Roboflow workspace...
loading Roboflow project...
⬇️ Downloading student-b3qgz/behaviordetector...
loading Roboflow workspace...
loading Roboflow project...


In [19]:
# ==========================================
# 3. PROCESS & MERGE (FULL FIXED VERSION)
# ==========================================
def normalize(name):
    return name.lower().replace("-", "").replace("_", "").replace(" ", "")

def process_dataset(source_path, target_class, prefix, limit=None):
    if not source_path:
        return 0

    # Load class names
    with open(f"{source_path}/data.yaml") as f:
        names = yaml.safe_load(f)["names"]

    # =============================
    # Detect numeric class names
    # =============================
    if all(n.isdigit() for n in names):
        print(f"   ⚡ Numeric class names detected in {source_path}, forcing mapping for target {target_class}")
        source_id = 0  # phone dataset is class 0
    else:
        # keyword-based matching for string names
        source_id = None
        for i, name in enumerate(names):
            norm_name = normalize(name)
            for kw in TARGET_CLASSES[target_class]:
                if kw in norm_name:
                    source_id = i
                    break

    if source_id is None:
        print(f"⚠️ No matching class found in {source_path}")
        print(f"   Available classes: {names}")
        return 0
    else:
        print(f"   ℹ️ Mapping '{names[source_id]}' → target {target_class}")

    # =============================
    # Process images
    # =============================
    images = glob(f"{source_path}/train/images/*")
    random.shuffle(images)

    count = 0
    for img in images:
        if limit and count >= limit:
            break

        base = os.path.splitext(os.path.basename(img))[0]
        lbl_path = f"{source_path}/train/labels/{base}.txt"
        if not os.path.exists(lbl_path):
            continue

        with open(lbl_path) as f:
            lines = f.readlines()

        new_lines = []
        for line in lines:
            cls, *bbox = line.split()
            if int(cls) == source_id:
                new_lines.append(f"{target_class} {' '.join(bbox)}\n")

        if not new_lines:
            continue

        new_img = f"{prefix}_{base}.jpg"
        new_lbl = f"{prefix}_{base}.txt"

        shutil.copy(img, f"{FINAL_DIR}/train/images/{new_img}")
        with open(f"{FINAL_DIR}/train/labels/{new_lbl}", "w") as f:
            f.writelines(new_lines)

        count += 1

    print(f"✅ Added {count} images from {prefix}")
    return count

# ==========================================
# RUN MERGE FOR PUBLIC DATASETS
# ==========================================
print("\n⚙️ Merging datasets...")
process_dataset(paths["phone"], 0, "phone", MAX_IMAGES)
process_dataset(paths["food"], 1, "food", MAX_IMAGES)
process_dataset(paths["drink"], 2, "drink", MAX_IMAGES)

# ==========================================
# PERSONAL DATA (SAFE REMAP)
# ==========================================
if paths["personal"]:
    print("⚙️ Processing personal dataset")

    with open(f"{paths['personal']}/data.yaml") as f:
        p_names = yaml.safe_load(f)["names"]

    # Normalize personal dataset names
    def normalize_personal(name):
        return name.lower().replace("-", "").replace("_", "").replace(" ", "")

    name_to_target = {normalize_personal(name): i for i, name in enumerate(['phone', 'food', 'drink'])}

    imgs = glob(f"{paths['personal']}/train/images/*")
    for img in imgs:
        base = os.path.splitext(os.path.basename(img))[0]
        lbl = f"{paths['personal']}/train/labels/{base}.txt"
        if not os.path.exists(lbl):
            continue

        with open(lbl) as f:
            lines = f.readlines()

        new_lines = []
        for line in lines:
            cls, *bbox = line.split()
            cname = normalize_personal(p_names[int(cls)])
            if cname in name_to_target:
                new_lines.append(f"{name_to_target[cname]} {' '.join(bbox)}\n")

        if not new_lines:
            continue

        shutil.copy(img, f"{FINAL_DIR}/train/images/personal_{base}.jpg")
        with open(f"{FINAL_DIR}/train/labels/personal_{base}.txt", "w") as f:
            f.writelines(new_lines)

    print("✅ Personal data merged")



⚙️ Merging datasets...
   ⚡ Numeric class names detected in /content/Phone-Detection-1, forcing mapping for target 0
   ℹ️ Mapping '0' → target 0
✅ Added 200 images from phone
   ℹ️ Mapping 'eating-person' → target 1
✅ Added 200 images from food
   ℹ️ Mapping 'drink-water' → target 2
✅ Added 200 images from drink
⚙️ Processing personal dataset
✅ Personal data merged


In [20]:
# ==========================================
# 3. MERGE & REMAP LOGIC
# ==========================================
# Target Classes: 0=phone, 1=food, 2=drink

def process_dataset(source_path, target_class_id, limit=200):
    if not source_path: return

    # Inspect data.yaml to find the source class ID
    with open(os.path.join(source_path, "data.yaml"), 'r') as f:
        data = yaml.safe_load(f)
        names = data['names']

    # Auto-find the best matching class ID in the source
    # (e.g., if source has ['head', 'phone'], we want ID 1)
    source_id = 0
    keywords = {
        0: ['phone', 'mobile', 'cell', 'smart'], # Target 0 (Phone)
        1: ['food', 'eat', 'burger', 'sandwich'], # Target 1 (Food)
        2: ['drink', 'cup', 'bottle', 'water']    # Target 2 (Drink)
    }

    # Simple search for the right ID
    for idx, name in enumerate(names):
        if any(k in name.lower() for k in keywords[target_class_id]):
            source_id = idx
            break

    print(f"   ℹ️ Mapping '{names[source_id]}' (ID {source_id}) -> Target ID {target_class_id}")

    # Process Images
    images = glob(os.path.join(source_path, "train", "images", "*"))
    random.shuffle(images)

    count = 0
    for img_path in images:
        if count >= limit: break

        # Find corresponding label
        basename = os.path.basename(img_path)
        name_only = os.path.splitext(basename)[0]
        label_file = os.path.join(source_path, "train", "labels", name_only + ".txt")

        if not os.path.exists(label_file): continue

        # Read label file
        with open(label_file, 'r') as f:
            lines = f.readlines()

        new_lines = []
        has_item = False
        for line in lines:
            parts = line.split()
            cls = int(parts[0])

            # If this line matches our desired object, remap and save
            if cls == source_id:
                new_line = f"{target_class_id} " + " ".join(parts[1:]) + "\n"
                new_lines.append(new_line)
                has_item = True

        if has_item:
            # Copy image
            shutil.copy(img_path, os.path.join(FINAL_DIR, "train", "images", basename))
            # Write new label
            with open(os.path.join(FINAL_DIR, "train", "labels", name_only + ".txt"), "w") as f:
                f.writelines(new_lines)
            count += 1

    print(f"   ✅ Added {count} images.")

print("\n⚙️ Processing Data...")
# Process Public Data (200 each)
process_dataset(path_phone, target_class_id=0, limit=MAX_IMAGES)
process_dataset(path_food, target_class_id=1, limit=MAX_IMAGES)
process_dataset(path_drink, target_class_id=2, limit=MAX_IMAGES)

# Process PERSONAL Data (Take ALL of it, no limit)
if path_personal:
    print("   ℹ️ Adding Personal Data (All)...")
    # Assuming personal data is already 0=phone, 1=food, 2=drink
    # If uncertain, we could map it, but usually personal data is the gold standard.
    p_imgs = glob(os.path.join(path_personal, "train", "images", "*"))
    for img in p_imgs:
        shutil.copy(img, os.path.join(FINAL_DIR, "train", "images", os.path.basename(img)))

    p_lbls = glob(os.path.join(path_personal, "train", "labels", "*"))
    for lbl in p_lbls:
        shutil.copy(lbl, os.path.join(FINAL_DIR, "train", "labels", os.path.basename(lbl)))
    print(f"   ✅ Added {len(p_imgs)} personal images.")



⚙️ Processing Data...
   ℹ️ Mapping '0' (ID 0) -> Target ID 0
   ✅ Added 200 images.
   ℹ️ Mapping 'eating-person' (ID 0) -> Target ID 1
   ✅ Added 200 images.
   ℹ️ Mapping 'drink-water' (ID 0) -> Target ID 2
   ✅ Added 200 images.
   ℹ️ Adding Personal Data (All)...
   ✅ Added 119 personal images.


In [21]:
from glob import glob

# Path to merged training labels
train_labels_path = f"{FINAL_DIR}/train/labels"

# Initialize counters
class_counts = {0: 0, 1: 0, 2: 0}  # 0=phone, 1=food, 2=drink

# Loop over all label files
for lbl_file in glob(f"{train_labels_path}/*.txt"):
    with open(lbl_file) as f:
        for line in f:
            cls = int(line.split()[0])
            if cls in class_counts:
                class_counts[cls] += 1

# Print results
print("📊 Total labels per class:")
for cls_id, count in class_counts.items():
    name = ['phone', 'food', 'drink'][cls_id]
    print(f"  {name} ({cls_id}): {count} annotations")

# Optional: count unique images per class
images_per_class = {0: set(), 1: set(), 2: set()}
for lbl_file in glob(f"{train_labels_path}/*.txt"):
    base = os.path.splitext(os.path.basename(lbl_file))[0]
    with open(lbl_file) as f:
        for line in f:
            cls = int(line.split()[0])
            if cls in images_per_class:
                images_per_class[cls].add(base)

print("\n📊 Total unique images per class:")
for cls_id, imgs in images_per_class.items():
    name = ['phone', 'food', 'drink'][cls_id]
    print(f"  {name} ({cls_id}): {len(imgs)} images")


📊 Total labels per class:
  phone (0): 592 annotations
  food (1): 601 annotations
  drink (2): 586 annotations

📊 Total unique images per class:
  phone (0): 445 images
  food (1): 573 images
  drink (2): 580 images


In [22]:
# ==========================================
# 4. TRAIN (UPDATED)
# ==========================================
import os, shutil, random
from glob import glob
from ultralytics import YOLO

# -------------------------------
# 1️⃣ Prepare train/val split (80/20)
# -------------------------------
train_dir = os.path.join(FINAL_DIR, 'train')
val_dir = os.path.join(FINAL_DIR, 'valid')

# Create valid folders
os.makedirs(os.path.join(val_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'labels'), exist_ok=True)

# Get all training images
all_images = glob(os.path.join(train_dir, 'images', '*.jpg'))
random.shuffle(all_images)
split_idx = int(0.8 * len(all_images))

train_imgs = all_images[:split_idx]
val_imgs = all_images[split_idx:]

# Move validation images and labels
for img_path in val_imgs:
    base = os.path.basename(img_path)
    lbl_path = os.path.join(train_dir, 'labels', base.replace('.jpg', '.txt'))

    shutil.move(img_path, os.path.join(val_dir, 'images', base))
    if os.path.exists(lbl_path):
        shutil.move(lbl_path, os.path.join(val_dir, 'labels', base.replace('.jpg', '.txt')))

print(f"📊 Training images: {len(train_imgs)}, Validation images: {len(val_imgs)}")

# -------------------------------
# 2️⃣ Create data.yaml
# -------------------------------
yaml_content = f"""
train: {os.path.join(train_dir, 'images')}
val: {os.path.join(val_dir, 'images')}
nc: 3
names: ['phone', 'food', 'drink']
"""
with open(os.path.join(FINAL_DIR, "data.yaml"), "w") as f:
    f.write(yaml_content)

print("\n🔥 Starting Training...")

# -------------------------------
# 3️⃣ Train YOLOv8 model
# -------------------------------
model = YOLO('yolov8n.pt')  # use small model for Colab T4

results = model.train(
    data=os.path.join(FINAL_DIR, "data.yaml"),
    epochs=50,
    imgsz=640,
    batch=16,
    name='behavior_model_final',
    patience=5,       # early stop if val loss doesn't improve
    save_period=5     # save checkpoints every 5 epochs
)


📊 Training images: 1313, Validation images: 329

🔥 Starting Training...
Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/merged_dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=behavior_model_final, nbs=64, nms=False,